<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_shiftLab_NLP_%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B9_%D0%95%D0%BC%D0%B5%D0%BB%D1%8C%D1%8F%D0%BD%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Будем разрабатывать текстового чат-бота, который помогает студентам НГУ с организационными вопросами.

#### **Содержание**:
1. Подготовительный этап
2. Исследование данных
3. Подготовка данных для обучения
4. Обучение классификатора
5. Деплой

Если какой-то из пунктов вызывает затруднения, его можно пропустить и приступить к следующему.

# 1. Подготовительный этап:




**1.1 Скопируем ноутбук на свой gdrive**

1.go to File-->Save copy in Drive

2.В открывшемся новом colab:

переименуйте название следующим образом "Тестовое задание shiftLab NLP + [ваше имя фамилия].ipynb"

например, "Тестовое задание shiftLab NLP Иван Иванов.ipynb"

**1.2 Подключим свой gdrive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1.2. Далее необходимо скачать архив с данными и положить в свой gdrive**

Ссылка на архив: https://drive.google.com/file/d/1_erk83F6up10pAjgFygGXr1GLxabSU6m/view?usp=sharing


**1.3. Далее нужно записать путь до папки с файлами в переменную PATH_TO_DATA**

In [ ]:
PATH_TO_DATA = '/content/drive/MyDrive/Test_SHIFTLAB_NLP/Unzipped_Files'

# 2. Исследование данных

Прежде всего, давайте считаем данные с диска и посмотрим как они выглядят

## 2.1 Вспомогательные функции:

Данные функции нужны для считывания файла с описанием интентов и отрисовки графиков

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_with_subplots(data, xlabel, ylabel, title):
    """
    Эта функция для отрисовки графиков
    """
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"colspan": 2}, None], [{}, {}]],
        subplot_titles=(title,"Head", "Tail")
    )
    
    data_head = data.head(10)
    data_tail = data.tail(10)

    bar_1 = go.Bar(y=data[xlabel], x=data.index)
    bar_2 = go.Bar(y=data_head[xlabel], x=data_head[ylabel])
    bar_3 = go.Bar(y=data_tail[xlabel], x=data_tail[ylabel])


    fig['layout']['xaxis2'].update(tickangle=-45)
    fig['layout']['xaxis3'].update(tickangle=-45)

    fig.add_trace(bar_1, row=1, col=1)
    fig.add_trace(bar_2, row=2, col=1)
    fig.add_trace(bar_3, row=2, col=2)

    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
import re

DESC_PATTERN = '(\w+(?:\s+\w+)*)\s*-\s*(\w+(?:\s+\w+)*)'

def parse_labels_description(desc_path):
    """
    Эта функция парсит файлик с описанием интентов
    """
    with open(desc_path, 'r') as desc:
        pattern = re.compile(DESC_PATTERN)

        labels_descriptions = []
        for line in desc.readlines():
            labels_descriptions.append(re.findall(pattern, line)[0])

        labels_descriptions = pd.DataFrame(labels_descriptions, columns=['intent', 'desc'])
    return labels_descriptions

## 2.2 Базовая информация по датасету:

In [ ]:
# Парсим описания интентов:
import pandas as pd
import os

LABELS_DESC_PATH = os.path.join(PATH_TO_DATA, 'labels_description.txt')

labels_descriptions = parse_labels_description(LABELS_DESC_PATH)
labels_descriptions.head(10)

,intent,desc
0,statement_general,get a document of some undefined type
1,incomestat,get an income statement
2,conform,get a document that proofs that a person belon...
3,transcript,get a student transcript
4,loc_cafeteria_new_building_general,find a cafe in building
5,loc_cafeteria_new_building_1_etage,find a cafe in building
6,loc_cafeteria_new_building_2_etage,find a cafe in building
7,loc_cafeteria_new_building_3_etage,find a cafe in building
8,loc_cafeteria_new_building_4_etage,find a cafe in building
9,sched_general,get a schedule of classes


In [ ]:
# Считывем данные:
import pandas as pd

DATASET_PATH = os.path.join(PATH_TO_DATA, 'dataset_train.tsv')

train_data = pd.read_csv(DATASET_PATH, delimiter='\t', encoding="utf-8", names=['text', 'intent'])
train_data.sample(10)

,text,intent
11351,расписание для Быкадоров И.А.,sched_teacher
13170,оформлять финансирование на поездку,travelfinaid_get
4285,план корпусов где есть,campus_map
7574,стоять мца,math_center_def
6862,заказать стипендия повышенная!,highscholarship
4673,погода в академгородке,weather
5833,заказать справка студента эф!,nsunet
8743,понедельник 487930,sched_for_group_day
5498,продуктовый распологается где.,loc_shop
5703,расположен где мца?,math_center_def


In [ ]:
# Размер датасета:

train_data.shape

(13230, 2)

In [ ]:
# Количество интентов / классов:

train_data.intent.nunique()

142

## 2.3 Распределение интентов:

In [ ]:
# Давайте посмотрим как выглядит распределение интентов в данных:

intents_distribution = (
    train_data
    .groupby('intent', as_index=False)
    .agg(len)
    .sort_values(by='text', ascending=False)
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text': 'n_samples'})
)
intents_distribution.head(10)

,intent,n_samples,desc
0,sched_teacher,1110,schedule of a teacher
1,sched_for_group,405,get a schedule for a student group
2,sched_for_group_day,402,get a schedule for a student group on a specif...
3,wifi,287,how to get wifi in a campus
4,status_free,271,how to leave a university
5,loc_dean_mmf,264,where is a dean of the mathematics faculty
6,loc_shop,237,find out a convinient shop
7,stat_numb_of_students,219,how many students there are in the university
8,nsunet,211,get the internet in a dormitory
9,student_union_event_guests,203,how a student union can invite guests


In [ ]:
# Нарисуем распределение
# Нижние графики показывают начало (HEAD)  и хвост (TAIL) распределения

plot_with_subplots(intents_distribution, 'n_samples', 'intent', 'Распределение интентов')

## 2.4 Распределение min, max, mean длин текстов:

2.4.1 распределение минимальных длин

In [ ]:
# Давайте посмотрим как выглядит распределение минимальных длин текстов:

min_len_distribution = (
    train_data
    .assign(text_len = train_data.text.str.len())
    .groupby('intent', as_index=False)
    .text_len
    .agg(min)
    .sort_values(by='text_len', ascending=False)
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text_len': 'min_text_len'})
)

In [ ]:
# Нарисуем распределение
# Нижние графики показывают начало (HEAD)  и хвост (TAIL) распределения

plot_with_subplots(min_len_distribution, 'min_text_len', 'intent', 'Распределение минимальных длин')

2.4.2 Распределение максимальных длин

In [ ]:
# Требуется аналогично приведенному выше варианту вывести график распределения максимальных длин

#max_len_distribution =  Your code

In [ ]:
max_len_distribution = (
    train_data
    .assign(text_len = train_data.text.str.len())
    .groupby('intent', as_index=False)
    .text_len
    .agg(max)
    .sort_values(by='text_len', ascending=False)
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text_len': 'max_text_len'})
)

In [ ]:
plot_with_subplots(max_len_distribution, 'max_text_len', 'intent', 'Распределение максимальных длин')

2.4.3 Распределение средних длин

In [ ]:
# Требуется аналогично приведенному выше варианту вывести график распределения средних длин

#mean_len_distribution =  Your code

In [ ]:
import numpy as np

mean_len_distribution = (
    train_data
    .assign(text_len = train_data.text.str.len())
    .groupby('intent', as_index=False)
    .text_len
    .agg(np.mean)
    .sort_values(by='text_len', ascending=False)
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text_len': 'mean_text_len'})
)

In [ ]:
plot_with_subplots(mean_len_distribution, 'mean_text_len', 'intent', 'Распределение средних длин')

2.4.4 Какие интенты попали в пересечение HEAD минимальных, максимальных и средних длин?

In [ ]:
min_len_distribution

,intent,min_text_len,desc
0,student_union_room,45,how to get a meeting room for a student union
1,loc_dean_med,39,where is a dean of the medical faculty
2,loc_cytologygeneticsinstitute,36,find out a cytology and genetics institute
3,loc_compcenter,33,find out a computational center
4,loc_cafeteria_new_building_3_etage,28,find a cafe in building
...,...,...,...
137,studentorgs_define,2,what students organization there are
138,smalltalk_greetings,2,greetings
139,loc_torgovy_center,2,find out a shopping mall
140,loc_dean_ef,2,where is a dean of the economics faculty


In [ ]:
#intersection = # Your code

In [ ]:
intersection = list(set(min_len_distribution['intent'][:10]) &
                    set(max_len_distribution['intent'][:10]) &
                    set(mean_len_distribution['intent'][:10]))

In [ ]:
intersection

['loc_cafeteria_new_building_4_etage',
 'student_union_room',
 'loc_compcenter',
 'loc_dean_med']

## 2.5 Поиск самых частых и редких слов:

2.5.1 Самы частые слова:

In [ ]:
def find_most_frequent_words(group, top_n):
    """
    Эта функция возвращает через запятую top_n самых частых слов для выбранного интента
    """
    group_words = sum(group.text.str.split().values, [])
    value_counts = pd.Series(group_words).value_counts()
    top_n_values = value_counts.iloc[:top_n]

    return ', '.join(top_n_values.index)

In [ ]:
top_n = 5

words_freq_distribution = (
    train_data
    .groupby('intent', as_index=False)
    .agg(lambda group: find_most_frequent_words(group, top_n))
    .sort_values(by='text')
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text': 'most_frequent_words'})
)

In [ ]:
words_freq_distribution 

,intent,most_frequent_words,desc
0,student_union_zoom,"zoom, как, получить, конференция, встреча",get a zoom romm for a student union
1,smalltalk_greetings,"Привет,, добрый, привет, Добрый, день",greetings
2,status_akadem,"академ, академотпуск, отпуск, академический, как",how to make a break from the university
3,stdscholarship,"академическая, стипендия, государственная, гас...",how to get a standard state scholarship
4,loc_pharm,"аптекарь, аптечка, аптека, где, расположен",find out a pharmacy
...,...,...,...
137,student_union_chorus,"хор, где, академический, нгу, находится",how to join a student chorus
138,loc_cafeteria_new_building_1_etage,"этаж, корпус, 1, первый, где",find a cafe in building
139,loc_cafeteria_new_building_2_etage,"этаж, корпус, второй, 2, где",find a cafe in building
140,loc_cafeteria_new_building_3_etage,"этаж, корпус, третий, 3, где",find a cafe in building


2.5.2 Самые редкие слова:

In [ ]:
#  Похожим образом найти самые редкие слова.

# words_freq_distribution = Your code

In [ ]:
def find_less_frequent_words(group, top_n):
    """
    Эта функция возвращает через запятую top_n самых редких слов для выбранного интента
    """
    group_words = sum(group.text.str.split().values, [])
    value_counts = pd.Series(group_words).value_counts()
    top_n_values = value_counts.iloc[-top_n:]

    return ', '.join(top_n_values.index)

In [ ]:
top_n = 5

words_less_freq_distribution = (
    train_data
    .groupby('intent', as_index=False)
    .agg(lambda group: find_less_frequent_words(group, top_n))
    .sort_values(by='text')
    .merge(labels_descriptions, on='intent', how='left')
    .rename(columns={'text': 'less_frequent_words'})
)

In [ ]:
words_less_freq_distribution

,intent,less_frequent_words,desc
0,sched_for_group_day,"21238, 20644, 715132, 594299, 224663",get a schedule for a student group on a specif...
1,sched_for_group,"22409, 20687, 21335, 22097, 220840",get a schedule for a student group
2,sched_teacher,"Дмитриева, Орловская, Пригарин, Пальчикова, Панов",schedule of a teacher
3,student_union_event_room,"актовый, для, мероприятий, помещение, цввр",how to get a room for a student union event
4,diploma_red_def,"бы, хотеть, есть, получать, получаться",how to get a diploma with excellence
...,...,...,...
137,trade_union_general,"университетские, объединение, союзам, професси...",how to join a union of any type
138,stat_numb_of_students,"учиться, заниматься, штудировать, усваивать, гдео",how many students there are in the university
139,smalltalk_greetings,"хаю, рад, Витя,, поговорить, Кеша,",greetings
140,statement_general,"чорт, и, справочку, надо, нужно",get a document of some undefined type


2.5.3 Найти имена всех преподавателей НГУ

In [ ]:
# Так как это чат-бот НГУ, то в данных есть информация о расписании каждого преподавателя, давайте попробуем найти имена этих преподавателей.

#names =  Your code

In [ ]:
pip install natasha

     |████████████████████████████████| 34.4 MB 29 kB/s 
     |████████████████████████████████| 41 kB 132 kB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 8.2 MB 23.6 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=4f29870b5402e77583f4fd0a20be308111b1fece6e006b0f766fb98e5807d503
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [ ]:
lst = []
for string in train_data['text']:
  doc = Doc(string)
  doc.segment(segmenter) 
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  
  for span in doc.spans:
      span.normalize(morph_vocab)

  names_dict = {_.text: _.normal for _ in doc.spans if _.text != _.normal}
  if (names_dict):
    lst.append(names_dict)

lst

[{'Толкачева Е.М.': 'Толкачев Е.М.'},
 {'Барчунова Т.В.': 'Барчунов Т.В.'},
 {'Снытникова Н.И.': 'Снытников Н.И.'},
 {'Щеглова О.Г.': 'Щеглов О.Г.'},
 {'Ческидова Н.В.': 'Ческидов Н.В.'},
 {'Чернова Н.И.': 'Чернов Н.И.'},
 {'Максимова О.Д.': 'Максимов О.Д.'},
 {'Сухинина И.А.': 'Сухинин И.А.'},
 {'Тапилина Е.А.': 'Тапилин Е.А.'},
 {'Анисимова Н.К.': 'Анисимов Н.К.'},
 {'Саблина': 'Саблин'},
 {'Билута П.А.': 'Билут П.А.'},
 {'Попова Н.А.': 'Попов Н.А.'},
 {'Тарабунова Н.В.': 'Тарабунов Н.В.'},
 {'Комарова А.В.': 'Комаров А.В.'},
 {'Притчина Е.А.': 'Притчин Е.А.'},
 {'Пухначева Т.П.': 'Пухначев Т.П.'},
 {'Симонова Е.В.': 'Симонов Е.В.'},
 {'Фролова Е.Л.': 'Фролов Е.Л.'},
 {'Туманова Н.Б.': 'Туманов Н.Б.'},
 {'Талышева Л.П.': 'Талышев Л.П.'},
 {'Халимская Л.М.': 'Халимскай Л.М.'},
 {'Шатрова В.Я.': 'Шатров В.Я.'},
 {'Назинцева Л.Н.': 'Назинцев Л.Н.'},
 {'Малинина Е.Е.': 'Малинин Е.Е.'},
 {'Пиков Г.Г.': 'Пики Г.Г.'},
 {'Сагдеева Л.Э.': 'Сагдеев Л.Э.'},
 {'Бугаева С.Г.': 'Бугаев С.Г.'},
 {'

# 3. Подготовка данных для обучения:

## 3.1 Пример подготовки данных для обучения:

In [ ]:
# Разделим обучающую выборку:

from sklearn.model_selection import train_test_split

X = train_data.text.values
y = train_data.intent

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
# Векторизуем корпус:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# Подготовим таргет

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

## 3.2 Ваш вариант подготовки данных для обучения:

In [ ]:
# Проявите фантазию и предложите альтернативные методы векторизации, которые дадут лучшее качетсво на классификации

#X_train, X_test, y_train, y_test =  Your code

## Предобработка

In [ ]:
import string
import re
import nltk
from nltk.corpus import stopwords
from string import punctuation

In [ ]:
import numpy as np
from tqdm.auto import tqdm, trange

In [ ]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)    

In [ ]:
data = train_data

In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(data['text'])]

  0%|          | 0/13230 [00:00<?, ?it/s]

In [ ]:
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(prep_text):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

data['text_sw'] = sw_texts_list

  0%|          | 0/13230 [00:00<?, ?it/s]

In [ ]:
pip install pymorphy2

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    str1 = ' '.join(res)    

    return str1

In [ ]:
lemm_texts_list = []
for text in tqdm(prep_text):
  lemm_texts_list.append(lemmatize(text))

data['text_lemm'] = lemm_texts_list

  0%|          | 0/13230 [00:00<?, ?it/s]

In [ ]:
data.head(10)

,text,intent,text_sw,text_lemm
0,мне нужна справка,statement_general,нужна справка,я нужный справка
1,оформить справку,statement_general,оформить справку,оформить справка
2,взять справку,statement_general,взять справку,взять справка
3,справку как получить,statement_general,справку получить,справка как получить
4,справку ммф где получаться,statement_general,справку ммф получаться,справка ммф где получаться
5,справка как получить,statement_general,справка получить,справка как получить
6,справку оформить,statement_general,справку оформить,справка оформить
7,оформлять справка,statement_general,оформлять справка,оформлять справка
8,оформлять справку,statement_general,оформлять справку,оформлять справка
9,справку,statement_general,справку,справка


In [ ]:
X = data['text_lemm']
y = data['intent']

# 4. Обучение классификатора:

## 4.1 Пример обучения классификатора:

In [ ]:
# Обучим Random Forest

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=300, random_state=0, n_estimators=50)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=300, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
# Сделаем предсказание

prediction = clf.predict(X_test)

In [ ]:
# Оценим качество на отложенных данных

from sklearn.metrics import f1_score

f1_score(y_test, prediction, average='micro')

0.9191232048374905

## 4.2 Ваш вариант обучения классификатора

In [ ]:
# Проявите фантазию и предложите альтернативные методы классификации, возможно нейросетевые, проведети настройку параметров для достижения лучшего качества

# Your code

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features = 1000, min_df=5, max_df=0.9)),
    ('sgd_clf', SGDClassifier(random_state=42))])

In [ ]:
parameters = { 
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=5, n_jobs=-1).fit(X_train, y_train)
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)


Best score and parameter combination:
0.9264923077472581 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'modified_huber', 'sgd_clf__penalty': 'l2'}


In [ ]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(penalty='l2', class_weight=None, loss = 'modified_huber', random_state=42))])
sgd_ppl_clf.fit(X_train, y_train)
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

                                     precision    recall  f1-score   support

                    advice_learning       0.89      1.00      0.94         8
                        advice_okno       1.00      1.00      1.00        21
                         campus_map       1.00      1.00      1.00        40
                            conform       1.00      0.88      0.94        43
                        cvvr_define       1.00      1.00      1.00         3
                    diploma_red_def       1.00      1.00      1.00        12
            dorm_enter_without_pass       1.00      1.00      1.00        12
                        dorm_guests       1.00      1.00      1.00        23
                        dorm_living       1.00      1.00      1.00        12
                          dorm_pets       1.00      1.00      1.00         6
                         dorm_price       1.00      1.00      1.00         6
                      dormcard_lost       1.00      1.00      1.00        1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



# 5. Деплой классификатора на хостинге Heroku с интерфейсом в Телеграм

На этом шаге мы хотим сделать телеграм бота, который будет классифицировать обращения и возвращать описание интента.

## 5.1 Научимся сохранять обученные модели sklearn

In [ ]:
import _pickle as cPickle

def save_model(model, filename):
    with open(filename, 'wb') as fid:
        cPickle.dump(model, fid)

def load_model(filename):
    with open(filename, 'rb') as fid:
        model = cPickle.load(fid)
    return model

In [ ]:
# Сохраняем:

save_model(clf, './classifier.pkl')
save_model(vectorizer, './vectorizer.pkl')
save_model(le, './le.pkl')

In [ ]:
from sklearn.externals import joblib
joblib.dump(model.best_estimator_, 'pipeline.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



['pipeline.pkl']

In [ ]:
# Загружаем:

clf = load_model('./classifier.pkl')
vectorizer = load_model('./vectorizer.pkl')
le = load_model('./le.pkl')

In [ ]:
l_model = joblib.load('pipeline.pkl')


In [ ]:
# Делаем предсказание:

my_test_data  = [
                 'расписание ммф',
                 'оформить мат помощь',
                 'где находится мед пункт',
                 'ближайший банкомат'
]

my_test_data = vectorizer.transform(my_test_data)
predict_labels = clf.predict(my_test_data)
le.inverse_transform(predict_labels)

array(['loc_dean_mmf', 'finassist', 'loc_commissariat', 'loc_atm'],
      dtype=object)

In [ ]:
my_test_data  = [
                 'расписание ммф',
                 'оформить мат помощь',
                 'где находится мед пункт',
                 'ближайший банкомат'
]

predicted = l_model.predict(my_test_data)

In [ ]:
predicted

array(['loc_dean_mmf', 'finassist', 'loc_commissariat', 'loc_atm'],
      dtype='<U35')

## 5.2 Создание Телеграм бота на Python

Туториал - как создать телеграм бота

https://www.codementor.io/@karandeepbatra/part-1-how-to-create-a-telegram-bot-in-python-in-under-10-minutes-19yfdv4wrq

## 5.3 Деплой приложения на Heroku

Туториал - как задеплоить приложение на heroku

https://towardsdatascience.com/how-to-create-and-deploy-a-machine-learning-app-to-heroku-d6965aa4f627

# Контакты:

По всем возникающим вопросам писать в Телеграм @Evgeniy_Kurochkin, @kotinigor